<a href="https://colab.research.google.com/github/meta-llama/llama-stack/blob/main/docs/Tool_Calling101_With_Together_Llama_Stack_Server.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

If you'd prefer not to set up a local server, explore this on tool calling with the Together API. This guide will show you how to leverage Together.ai's Llama Stack Server API, allowing you to get started with Llama Stack without the need for a locally built and running server.

## Creating a Custom Tool and Agent Tool Calling with Together API


## Step 1: Import Necessary Packages and import api keys

In [6]:
import os
import requests
import json
import asyncio
import nest_asyncio
from typing import Dict, List
from dotenv import load_dotenv
from llama_stack_client import LlamaStackClient
from llama_stack_client.lib.agents.custom_tool import CustomTool
from llama_stack_client.types.shared.tool_response_message import ToolResponseMessage
from llama_stack_client.types import CompletionMessage
from llama_stack_client.lib.agents.agent import Agent
from llama_stack_client.lib.agents.event_logger import EventLogger
from llama_stack_client.types.agent_create_params import AgentConfig

# Allow asyncio to run in Jupyter Notebook
nest_asyncio.apply()

LLAMA_STACK_API_TOGETHER_URL="https://llama-stack.together.ai"

Create a `.env` file and add you brave api key

`BRAVE_SEARCH_API_KEY = "YOUR_BRAVE_API_KEY_HERE"`

Now load the `.env` file into your jupyter notebook.

In [7]:
load_dotenv()
BRAVE_SEARCH_API_KEY = os.environ['BRAVE_SEARCH_API_KEY']

## Step 2: Create a class for the Brave Search API integration

Let's create the `BraveSearch` class, which encapsulates the logic for making web search queries using the Brave Search API and formatting the response. The class includes methods for sending requests, processing results, and extracting relevant data to support the integration with an AI toolchain.

In [8]:
class BraveSearch:
    def __init__(self, api_key: str) -> None:
        self.api_key = api_key

    async def search(self, query: str) -> str:
        url = "https://api.search.brave.com/res/v1/web/search"
        headers = {
            "X-Subscription-Token": self.api_key,
            "Accept-Encoding": "gzip",
            "Accept": "application/json",
        }
        payload = {"q": query}
        response = requests.get(url=url, params=payload, headers=headers)
        return json.dumps(self._clean_brave_response(response.json()))

    def _clean_brave_response(self, search_response, top_k=3):
        query = search_response.get("query", {}).get("original", None)
        clean_response = []
        mixed_results = search_response.get("mixed", {}).get("main", [])[:top_k]

        for m in mixed_results:
            r_type = m["type"]
            results = search_response.get(r_type, {}).get("results", [])
            if r_type == "web" and results:
                idx = m["index"]
                selected_keys = ["title", "url", "description"]
                cleaned = {k: v for k, v in results[idx].items() if k in selected_keys}
                clean_response.append(cleaned)

        return {"query": query, "top_k": clean_response}

## Step 3: Create a custom tool class for integration with Llama Stack

Here, we defines the `WebSearchTool` class, which extends `CustomTool` to integrate the Brave Search API with Llama Stack, enabling web search capabilities within AI workflows. The class handles incoming user queries, interacts with the `BraveSearch` class for data retrieval, and formats results for effective response generation.

In [9]:
class WebSearchTool(CustomTool):
    def __init__(self, api_key: str):
        self.api_key = api_key
        self.engine = BraveSearch(api_key)

    def get_name(self) -> str:
        return "web_search"

    def get_description(self) -> str:
        return "Search the web for a given query"

    async def run_impl(self, query: str):
        return await self.engine.search(query)

    async def run(self, messages):
        query = None
        for message in messages:
            if isinstance(message, CompletionMessage) and message.tool_calls:
                for tool_call in message.tool_calls:
                    if 'query' in tool_call.arguments:
                        query = tool_call.arguments['query']
                        call_id = tool_call.call_id

        if query:
            search_result = await self.run_impl(query)
            return [ToolResponseMessage(
                call_id=call_id,
                role="ipython",
                content=self._format_response_for_agent(search_result),
                tool_name="brave_search"
            )]

        return [ToolResponseMessage(
            call_id="no_call_id",
            role="ipython",
            content="No query provided.",
            tool_name="brave_search"
        )]

    def _format_response_for_agent(self, search_result):
        parsed_result = json.loads(search_result)
        formatted_result = "Search Results with Citations:\n\n"
        for i, result in enumerate(parsed_result.get("top_k", []), start=1):
            formatted_result += (
                f"{i}. {result.get('title', 'No Title')}\n"
                f"   URL: {result.get('url', 'No URL')}\n"
                f"   Description: {result.get('description', 'No Description')}\n\n"
            )
        return formatted_result

## Step 4: Create a function to execute a search query and print the results

Now let's create the `execute_search` function, which initializes the `WebSearchTool`, runs a query asynchronously, and prints the formatted search results for easy viewing.

In [10]:
async def execute_search(query: str):
    web_search_tool = WebSearchTool(api_key=BRAVE_SEARCH_API_KEY)
    result = await web_search_tool.run_impl(query)
    print("Search Results:", result)

## Step 5: Run the search with an example query

In [11]:
query = "Latest developments in quantum computing"
asyncio.run(execute_search(query))

Search Results: {"query": "Latest developments in quantum computing", "top_k": [{"title": "Quantum Computing | Latest News, Photos & Videos | WIRED", "url": "https://www.wired.com/tag/quantum-computing/", "description": "Find the <strong>latest</strong> <strong>Quantum</strong> <strong>Computing</strong> news from WIRED. See related science and technology articles, photos, slideshows and videos."}, {"title": "Quantum Computing News -- ScienceDaily", "url": "https://www.sciencedaily.com/news/matter_energy/quantum_computing/", "description": "<strong>Quantum</strong> <strong>Computing</strong> News. Read the <strong>latest</strong> about the <strong>development</strong> <strong>of</strong> <strong>quantum</strong> <strong>computers</strong>."}]}


## Step 6: Run the search tool using an agent

Here, we setup and execute the `WebSearchTool` within an agent configuration in Llama Stack to handle user queries and generate responses. This involves initializing the client, configuring the agent with tool capabilities, and processing user prompts asynchronously to display results.

In [12]:
async def run_main(url=LLAMA_STACK_API_TOGETHER_URL, disable_safety: bool = False):
    # Initialize the Llama Stack client with the specified base URL
    client = LlamaStackClient(
        base_url=url,
    )

    # Configure input and output shields for safety (use "llama_guard" by default)
    input_shields = [] if disable_safety else ["llama_guard"]
    output_shields = [] if disable_safety else ["llama_guard"]

    # Define the agent configuration, including the model and tool setup
    agent_config = AgentConfig(
        model="Llama3.2-3B-Instruct",  
        instructions="""You are a helpful assistant that responds to user queries with relevant information and cites sources when available.""",
        sampling_params={
            "strategy": "greedy", 
            "temperature": 1.0, 
            "top_p": 0.9,  
        },
        tools=[
            {
                "function_name": "web_search",  # Name of the tool being integrated
                "description": "Search the web for a given query",
                "parameters": {
                    "query": {
                        "param_type": "str", 
                        "description": "The query to search for",
                        "required": True, 
                    }
                },
                "type": "function_call", 
            },
        ],
        tool_choice="auto",  
        tool_prompt_format="python_list", 
        input_shields=input_shields,
        output_shields=output_shields,
        enable_session_persistence=False, 
    )
    
    # Initialize custom tools (ensure `WebSearchTool` is defined earlier in the notebook)
    custom_tools = [WebSearchTool(api_key=BRAVE_SEARCH_API_KEY)]

    # Create an agent instance with the client and configuration
    agent = Agent(client, agent_config, custom_tools)
    
    # Create a session for interaction and print the session ID
    session_id = agent.create_session("test-session")
    print(f"Created session_id={session_id} for Agent({agent.agent_id})")

    response = agent.create_turn(
        messages=[
            {
                "role": "user",  
                "content": """What are the latest developments in quantum computing?""", 
            }
        ],
        session_id=session_id,  # Use the created session ID
    )

    # Log and print the response from the agent asynchronously
    async for log in EventLogger().log(response):
        log.print()

# Run the function asynchronously in a Jupyter Notebook cell
await run_main(disable_safety=False)

Created session_id=e4062b71-8034-40a8-8124-3410d9ab8ddc for Agent(27ab94ce-cfe8-4af5-a71b-4e8f4bef6434)
shield_call> No Violation
inference> [web_search(query='latest developments in quantum computing')]
shield_call> No Violation
CustomTool> Search Results with Citations:

1. Quantum Computing | Latest News, Photos & Videos | WIRED
   URL: https://www.wired.com/tag/quantum-computing/
   Description: Find the <strong>latest</strong> <strong>Quantum</strong> <strong>Computing</strong> news from WIRED. See related science and technology articles, photos, slideshows and videos.

2. Quantum Computing News -- ScienceDaily
   URL: https://www.sciencedaily.com/news/matter_energy/quantum_computing/
   Description: <strong>Quantum</strong> <strong>Computing</strong> News. Read the <strong>latest</strong> about the <strong>development</strong> <strong>of</strong> <strong>quantum</strong> <strong>computers</strong>.


